In [ ]:
# Process results
def ProcessResults(Results, Model):
    WriteSolution = False
    Optimal = False
    LimitStop = False
    Condition = Results.solver.termination_condition

    # Defer loading results until now, in case there is no solution to load
    if Condition == pyo.TerminationCondition.maxTimeLimit or Condition == pyo.TerminationCondition.maxIterations:
        LimitStop = True
        WriteSolution = True
        Model.solutions.load_from(Results)
    if (Results.solver.status == pyo.SolverStatus.ok) and (Condition == pyo.TerminationCondition.optimal):
        Optimal = True
        WriteSolution = True
        Model.solutions.load_from(Results)                                                  
    try:
        SolverData = Results.Problem._list
        SolutionLB = SolverData[0].lower_bound
        SolutionUB = SolverData[0].upper_bound
    except:
         SolutionLB = float('-inf')
         SolutionUB = float('inf')
    # Otherwise, no solution
        
    return WriteSolution, LimitStop, SolutionLB, SolutionUB

In [ ]:
# Write model output
def Output(Results, Model, WriteSolution, LimitStop, SolutionLB, SolutionUB):
    if Neos:
        SolveEngine = 'Neos, ' + pyo.value(Model.Engine)
    else:
        SolveEngine = pyo.value(Model.Engine)

    print('\n')
    print(Model.name, '\n')
    print('Solver:   ', SolveEngine)
    print('Status:   ', Results.solver.status)
    print('Condition:', Results.solver.termination_condition, '\n')
    
    print('Objective bounds')   # Indicate how close we are to a solution. Only for some local solvers
    print('----------------')
    print(f'Lower: {SolutionLB:9,.2f}')
    print(f'Upper: {SolutionUB:9,.2f}\n')
            
    if LimitStop:
        print('Stopped at time or iteration limit')
    if WriteSolution:
        print(f'Racks   = {Model.Racks():3,.0f}')   # Number of racks
        print(f'Shelves = {sum(pyo.value(Model.Shelf[s]) for s in Model.S):3,.0f}\n')   # Number of shelves per rack

        print('Shelves')   # Shelf specification
        print('Use   Size')
        print('----------')
        for s in Model.S:
            Row = ''
            if round(pyo.value(Model.Shelf[s]), 0) == 0:
                Row += ' -    '
            else:
                Row += ' 1    '
            Size = round(pyo.value(Model.ShelfHeights[s]), 0)
            if Size == 0:
                Row += '  -'
            else:
                Row += '{:3.0f}'.format(Size)
            print(Row)

        print('\nAllocation')   # Allocation of pallets to shelves
        Header =  '           Shelf\n'
        Header += 'Pallet  '
        for s in Model.S:
            Header += '{:6.0f}'.format(s + 1)
        Header += '\n--------' + 6 * len(Model.S) * '-'
        print(Header)
        for p in Model.P:
            Row = '{:6.0f}'.format(p + 1) + '  '
            for s in Model.S:
                if round(pyo.value(Model.Allocation[p, s]), 0) == 1:
                    Row += '     1'
                else:
                    Row += '     -'
            print(Row)
        Footer = '--------'  + 6 * len(Model.S) * '-' + '\n'
        TotalRow = 'Total    '   # Total pallets allocated to each shelf
        for s in Model.S:
            NumPallets = 0
            for p in Model.P:
                NumPallets += round(pyo.value(Model.Allocation[p, s]), 0)
            if NumPallets == 0:
                TotalRow += '    - '
            else:
                TotalRow += '{:5.0f}'.format(NumPallets) + ' '
        Footer += TotalRow
        print(Footer)
    else:
        print('No solution loaded\n')